In [1]:
pip install diagrams
pip install graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 25.1 MB/s eta 0:00:00


In [18]:

# mvp_architecture.py
# To generate the diagram, install Graphviz and diagrams:
# pip install diagrams
# sudo apt install graphviz

from diagrams import Diagram, Cluster, Edge
from diagrams.aws.network import Route53, CloudFront, VPC, PublicSubnet, NATGateway, APIGateway
from diagrams.aws.storage import S3
from diagrams.aws.database import Dynamodb
from diagrams.aws.compute import AppRunner
from diagrams.aws.security import ACM, Cognito, SecretsManager
from diagrams.aws.management import Cloudwatch
from diagrams.aws.general import User
from diagrams.onprem.ci import GithubActions
from diagrams.onprem.iac import Terraform

graph_attr = {
    "fontsize": "10",
    "bgcolor": "white"
}

cluster_attr = {
    "fontname": "Helvetica-Bold",
    "fontsize": "12",
    "bgcolor": "#F7F7F7",
    "pencolor": "#BDBDBD",
}

node_attr = {
    "fontname": "Helvetica",
    "fontsize": "10",
    "imagescale": "true",
    "fixedsize": "true",
    "width": "1.2",
    "height": "1.2",
}

with Diagram("EngageIQ MVP Architecture", show=False, filename="engageiq_mvp_architecture", graph_attr=graph_attr, direction="LR"):

    user = User("Browser User")

    waf_footnote = "Note: AWS WAF is optional for enhanced CloudFront security."
    sg_footnote = "App Runner SG can restrict inbound to AWS Edge IPs (CloudFront)."
    rag_placeholder = "Future: RAG Engine (e.g., ChromaDB / OpenSearch)"
    api_routes_label = (
        "MVP API Endpoints:\n"
        "- POST /submit-question\n"
        "- GET /get-chat-history\n"
        "- POST /ingest-customer-score\n"
        "- GET /get-ingestion-files"
    )

    # CI/CD + IaC
    with Cluster("CI/CD + IaC", graph_attr=cluster_attr):
        github = GithubActions("GitHub Actions\n(CI/CD)")
        terraform = Terraform("Terraform Cloud\n(Plan/Apply via OIDC)")
        cloudfront_invalidation = CloudFront("Cache Invalidation")

    # EDGE
    with Cluster("EDGE", graph_attr=cluster_attr):
        route53 = Route53("Route 53")
        cloudfront = CloudFront("CloudFront")
        acm_cert = ACM("ACM Certificate (TLS)")

    # AUTH
    with Cluster("AUTH", graph_attr=cluster_attr):
        cognito_ui = APIGateway("Hosted UI")
        cognito = Cognito("Cognito (User Pool)\n(org-xxx-users)")

    # SPA
    with Cluster("SPA", graph_attr=cluster_attr):
        s3_assets = S3("S3 Bucket\n(React SPA)")

    # APP HOSTING
    with Cluster("APP HOSTING", graph_attr=cluster_attr):
        with Cluster("VPC", graph_attr={"bgcolor": "transparent", "pencolor": "#87CEEB", "penwidth": "2"}):
            with Cluster("AZ A"):
                subnet_a = PublicSubnet("Public Subnet A")
            with Cluster("AZ B"):
                subnet_b = PublicSubnet("Public Subnet B")
            app_runner = AppRunner("App Runner\n(FastAPI Service)\nPublic Networking (MVP)")
            nat_gw = NATGateway("NAT GW (Future)")

    # DATA
    with Cluster("DATA", graph_attr=cluster_attr):
        dynamodb = Dynamodb("DynamoDB\nTenant Scoped\n(PK = ORG#123)\n(SK = USER#456)")
        s3_docs = S3("Docs Bucket\n/org-123/")

    # OBSERVABILITY
    with Cluster("OBSERVABILITY", graph_attr=cluster_attr):
        cloudwatch = Cloudwatch("CloudWatch\nLogs & Metrics")
        secrets = SecretsManager("Secrets Manager")

    # Arrows
    user >> Edge(label="1") >> route53
    route53 >> Edge(label="2\n*.engageiq.tech") >> cloudfront
    user << Edge(label="3. Redirect", style="dashed") >> cognito_ui
    cognito_ui >> Edge(label="JWT", style="dashed") >> user
    cognito_ui - cognito

    cloudfront >> Edge(label="4a. SPA") >> s3_assets
    cloudfront >> Edge(label="4b. API") >> app_runner

    app_runner >> Edge(label="5a", style="dotted") >> dynamodb
    app_runner >> Edge(label="5b", style="dotted") >> s3_docs
    app_runner >> Edge(label="6", style="dotted") >> cloudwatch
    app_runner >> Edge(label="Secrets", style="dotted") >> secrets

    github >> Edge(label="7a. Deploy") >> terraform
    terraform >> Edge(label="7b. Deploy SPA", style="dashed") >> s3_assets
    terraform >> Edge(label="7c. Deploy App", style="dashed") >> app_runner
    terraform >> Edge(label="7d. Invalidate", style="dashed") >> cloudfront_invalidation

    cloudfront - acm_cert
    app_runner - [subnet_a, subnet_b]

    # Notes and placeholders
    s3_docs >> Edge(label=rag_placeholder, style="dotted", color="gray")
    app_runner >> Edge(label=api_routes_label, style="dotted", color="gray")
    cloudfront >> Edge(label=waf_footnote, style="dotted", color="gray")
    app_runner >> Edge(label=sg_footnote, style="dotted", color="gray")
